In [ ]:
import boto3
import json
import pandas as pd
import re

s3 = boto3.client("s3")
bucket = "c144972a3751929l10036967t1w533267341463-bucket-pp6e047xcla7" #enter your bucket name here
transcript_prefix = "transcribe-job-"
job_ids = [
    "abc12345", "def67890"
]

def normalize(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

all_transcripts = []

for job_id in job_ids:
    key = f"{transcript_prefix}{job_id}.json"
    try:
        response = s3.get_object(Bucket=bucket, Key=key)
        data = json.loads(response["Body"].read())
        raw = data["results"]["transcripts"][0]["transcript"]
        norm = normalize(raw)
        all_transcripts.append({
            "job_id": job_id,
            "raw_text": raw,
            "normalized_text": norm
        })
        print(f"Downloaded: {key}")
    except Exception as e:
        print(f"Failed: {key} — {e}")
        all_transcripts.append({
            "job_id": job_id,
            "raw_text": None,
            "normalized_text": f"ERROR: {e}"
        })

df_transcripts = pd.DataFrame(all_transcripts)
df_transcripts.head()
